# Financial Markets - Practical Lesson 8

* Last lesson we implemented a `CreditCurve` class which allows us to interpolate pillar **non-default probabilities** (NDPs) at arbitrary value dates, and also allows us to calculate the forward **hazard rate** at an arbitrary date.

* This lesson we'll use that class to price **credit default swaps** (CDSs).


## Credit Default Swaps

CDSs are made up of two legs:

* The **default** leg: which pays $L = 1 - R$, known as the **loss given default** (LGD) if and when the credit event occurs, where $R = 40\%$ is a conventional **recovery value**.

* The **premium** leg: which pays $S$ every 3 months until the credit event occurs

### Premium leg

Let's start with the premium leg, which is easier to calculate. We'll use the following notation:

* $d$ today's date
* $d_0$ the start date of the CDS (could be different from $d$)
* $d_1, ..., d_n$ the payment dates of the premium leg, which occur at a 3-month frequency
* We assume that $d_n$ is the end date of the CDS.
* $D(d')$ the discount factor between $d$ and $d'$, i.e. the NPV in $d$ of a zero coupon bond expiring in $d'$
* $N(d')$ the non-default probability between $d$ and $d'$, i.e. the risk-neutral probability of the issuer *not* suffering a credit event before $d'$ as observed at date $d$.
* $\tau$ the random variable representing the date of the credit event

At each payment date $d_i$, a flow of $S$ is paid if and only if the credit event has not occurred before that date. Therefore the NPV of the each flow is

$$
\def\E{\mathbb{E}}
\def\1{\mathbb{1}}
\def\set#1{\!\left\{ #1 \right\}}
\tilde\E\left[ S \times D(d_i) \times \1\set{\tau > d_i} \right] = S \times D(d_i) \times N(d_i)
$$

therefore the NPV of the premium leg is simply the sum, over the payment dates, of these terms.

## Default leg

The LGD $1-R$ is paid out on the same date on which the credit event occurs, i.e. it can potentially be paid out on any date between $d_0$ and $d_n$. Mathematically, therefore, the NPV of the premium leg can be expressed as follows:

$$
\tilde\E[ (1-R) \times D(\tau) \times \1\set{\tau \leq d_n} ]
$$

Using the law of total probability, we can break this down in to the sum of "daily NPVs" calculated as a function of the daily forward default probabilities:

$$
\def\P{\mathbb{P}}
\begin{align*}
\tilde\E[ (1-R) \times D(\tau) \times \1\set{\tau \leq d_n} ]
&= \sum_{d'=d_0}^{d_n} \tilde\E[ (1-R) \times D(\tau) \mid \tau = d'] \tilde\P[ \tau = d' ] \\
&= (1-R) \sum_{d'=d_0}^{d_n} D(d') \left( \P[ \tau \geq d' ] - \P[ \tau \geq d'+1 ] \right) \\
&= (1-R) \sum_{d'=d_0}^{d_n} D(d') \left( N(d') - N(d'+1) \right)
\end{align*}
$$

In [30]:
from finmarkets import generate_swap_dates

class CreditDefaultSwap(object):
    
    def __init__(self, notional, start_date, nyears, fixed_spread, recovery=0.4):
        
        self.notional = notional
        self.payment_dates = generate_swap_dates(start_date, nyears*12, 3)
        self.fixed_spread = fixed_spread
        self.recovery = recovery
    
    def premium_leg_npv(self, discount_curve, credit_curve):
        
        npv = 0
        
        for i in range(1, len(self.payment_dates)):
            npv += (
                self.fixed_spread *
                discount_curve.df(self.payment_dates[i]) *
                credit_curve.ndp(self.payment_dates[i])
            )
            
        return npv * self.notional
    
    def default_leg_npv(self, discount_curve, credit_curve):
        
        npv = 0
        
        d = self.payment_dates[0]
        
        while d <= self.payment_dates[-1]:
            
            npv += discount_curve.df(d) * (
                credit_curve.ndp(d) -
                credit_curve.ndp(d + relativedelta(days=1))
            )
            
            d += relativedelta(days=1)
        
        return npv * self.notional * (1 - self.recovery)
    
    def npv(self, discount_curve, credit_curve):
        
        return (
            self.default_leg_npv(discount_curve, credit_curve) - 
            self.premium_leg_npv(discount_curve, credit_curve)
        )
        

In [31]:
from curve_data import discount_curve, pricing_date
from finmarkets import CreditCurve
from dateutil.relativedelta import relativedelta
credit_curve = CreditCurve([pricing_date, pricing_date + relativedelta(months=36)], [1.0, 0.7])

In [32]:
cds = CreditDefaultSwap(1e6, pricing_date, 3, 0.03)
cds.premium_leg_npv(discount_curve, credit_curve)

299885.15113920276

In [33]:
cds.default_leg_npv(discount_curve, credit_curve)

180853.40264428832

In [34]:
cds.npv(discount_curve, credit_curve)

-119031.74849491444